# Optimised with the help of chatgpt

In [4]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

path = 'Training_images'
images = []
classNames = []
myList = os.listdir(path)
print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)


def findEncodings(images):
    encodeList = []

    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(img)
        if len(face_locations) > 0:
            encode = face_recognition.face_encodings(img)[0]
            encodeList.append(encode)
    return encodeList


def markAttendance(name):
    with open('Attendance.xlsx', 'a') as f:
        now = datetime.now()
        dtString = now.strftime("%Y-%m-%d %H:%M:%S")
        f.write(f'\n{name},{dtString}')


encodeListKnown = findEncodings(images)
print('Encoding Complete')

cap = cv2.VideoCapture(0)

frame_count = 0
process_every_n_frame = 30  # Process every 5th frame

while True:
    success, img = cap.read()
    if not success:
        print("Failed to read frame from camera. Exiting...")
        break

    frame_count += 1
    if frame_count % process_every_n_frame != 0:
        continue  # Skip processing this frame

    imgS = cv2.resize(img, (0, 0), None, 0.5, 0.5)  # Reduce image size for smoother processing
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            markAttendance(name)
            cv2.putText(img, f'Attendance Marked: {name}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press '0' to exit the loop
        break

cap.release()
cv2.destroyAllWindows()


['Aayush.jpg']
['Aayush']
Encoding Complete
